In [1]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
import string

def hyperlinks_filter(tweet):
    # Removes hyperlinks from a given tweet
    hyperlink_pattern = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    t = re.sub(hyperlink_pattern, "", tweet)
    return t


def usernames_filter(tweet):
    # Removes usernames from a given tweet
    username_pattern = '(?<=^|(?<=[^a-zA-Z0-9-_\.]))@([A-Za-z_]+[A-Za-z0-9_]+)'
    t = re.sub(username_pattern, "", tweet)
    return t


def hashtags_filter(tweet):
    # Removes hashtags from a given tweet
    hashtag_pattern = '(?<=^|(?<=[^a-zA-Z0-9-\.]))#([A-Za-z0-9_]+[A-Za-z0-9_]+)'
    t = re.sub(hashtag_pattern, "", tweet)
    return t

def removePun(tweet):
    for c in string.punctuation:
        tweet= tweet.replace(c,"")
    return tweet


def tweet_filter(tweet, filters=[]):
    # Apply specified list of filter functions to a given tweet
    for f in filters:
        tweet = f(tweet)
    return tweet

def stop_words(tweet):
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(tweet)
    filtered_sentence = []
    for w in word_tokens:
        if w not in stop_words:
            filtered_sentence.append(w)
    return filtered_sentence


In [2]:
def check_sentiment(tweet):
    filters_apply = [hashtags_filter, hyperlinks_filter, usernames_filter, removePun ]
    n = 0
    p = 0
    s = tweet_filter(tweet,filters_apply)
    r = stop_words(s)
    for word in r:
        if word in negative_words:
            n += 1
        elif word in positive_words:
            p += 1
    if p > n:
        return 1
    elif p < n:
        return -1
    return 0

In [3]:
def is_valid(tweet):
    if "#playstation" in tweet.lower() and "#xbox" in tweet.lower():
        return False
    if "#playstation" in tweet.lower() and "#scratch" in tweet.lower():
        return False
    if "#scratch" in tweet.lower() and "#xbox" in tweet.lower():
        return False
    return True

In [4]:
with open('positive-words.txt') as f:
    positive_lines_after_intro = f.readlines()[30:]
with open('negative-words.txt') as f:
    negative_lines_after_intro = f.readlines()[31:]
positive_words = [line.rstrip() for line in positive_lines_after_intro]
negative_words = [line.rstrip() for line in negative_lines_after_intro]

In [5]:
import tweepy
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream

#Variables that contains the user credentials to access Twitter API
access_token = "1186393998023364610-6NHAzDONXHNNwlxGw2ggExSWssTjoq"
access_token_secret = "sY0lWtdmAvMKqyNc1pDPgKxYY2nzbCdU3nuZC8W4J9PbT"
consumer_key = "DQ6cYEFSD4Qqta1YmfIuy2DFW"
consumer_secret = "ndwWC678kC1aRnDVNAm4LVevHfFU42AdrnBuzgjcDdhGp7MNeD"

class StdOutListener(StreamListener):
    
    def __init__(self, api = None): 
        self.api = api
        self.play_station_pos = 0
        self.scratch_pos = 0
        self.xbox_pos = 0

        self.play_station_neg = 0
        self.scratch_neg = 0
        self.xbox_neg = 0

        self.most_pos = {"key": "",
                   "val": 0}
        self.most_neg = {"key": "",
                   "val": 0}

    def on_status(self, data):
        tweet = None
        try:
            tweet = data.extended_tweet["full_text"]
        except AttributeError:
            tweet = data.text        
        
        if tweet:
#             print(tweet)
            sentiment = check_sentiment(tweet)
            if "#playstation" in tweet.lower():
                if sentiment > 0:
                    self.play_station_pos += 1
                    if self.play_station_pos > self.most_pos['val']:
                        self.most_pos['key'] = "#PlayStation"
                        self.most_pos['val'] = self.play_station_pos
                if sentiment < 0:
                    self.play_station_neg += 1
                    if self.play_station_neg > self.most_neg['val']:
                        self.most_neg['key'] = "#PlayStation"
                        self.most_neg['val'] = self.play_station_neg

            elif "#xbox" in tweet.lower():
                if sentiment > 0:
                    self.xbox_pos += 1
                    if self.xbox_pos > self.most_pos['val']:
                        self.most_pos['key'] = "#Xbox"
                        self.most_pos['val'] = self.xbox_pos
                if sentiment < 0:
                    self.xbox_neg += 1
                    if self.xbox_neg > self.most_neg['val']:
                        self.most_neg['key'] = "#Xbox"
                        self.most_neg['val'] = self.xbox_neg

            elif "#scratch" in tweet.lower():
                if sentiment > 0:
                    self.scratch_pos += 1
                    if self.scratch_pos > self.most_pos['val']:
                        self.most_pos['key'] = "#Scratch"
                        self.most_pos['val'] = self.scratch_pos
                if sentiment < 0:
                    self.scratch_neg += 1
                    if self.scratch_neg > self.most_neg['val']:
                        self.most_neg['key'] = "#Scratch"
                        self.most_neg['val'] = self.scratch_neg
            log = "Most Postive: " + self.most_pos['key'] + " (" + str(self.most_pos['val']) + ") Most Negative: " + self.most_neg['key'] + " (" + str(self.most_neg['val']) + ")"
            print(log)

    def on_error(self, status):
        print(status)


In [6]:
l = StdOutListener()
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
stream = Stream(auth, l,  tweet_mode="extended")
stream.filter(track=["#Playstation", "#Xbox", "#Switch"])

Most Postive:  (0) Most Negative:  (0)
Most Postive:  (0) Most Negative:  (0)
Most Postive:  (0) Most Negative:  (0)
Most Postive:  (0) Most Negative:  (0)
Most Postive:  (0) Most Negative:  (0)
Most Postive:  (0) Most Negative:  (0)
Most Postive:  (0) Most Negative:  (0)
Most Postive: #Xbox (1) Most Negative:  (0)
Most Postive: #Xbox (1) Most Negative:  (0)
Most Postive: #Xbox (1) Most Negative:  (0)
Most Postive: #Xbox (1) Most Negative:  (0)
Most Postive: #Xbox (1) Most Negative:  (0)
Most Postive: #Xbox (1) Most Negative:  (0)
Most Postive: #Xbox (1) Most Negative:  (0)
Most Postive: #Xbox (1) Most Negative:  (0)
Most Postive: #Xbox (1) Most Negative:  (0)
Most Postive: #Xbox (1) Most Negative:  (0)
Most Postive: #Xbox (2) Most Negative:  (0)
Most Postive: #Xbox (2) Most Negative:  (0)
Most Postive: #Xbox (2) Most Negative:  (0)
Most Postive: #Xbox (2) Most Negative:  (0)
Most Postive: #Xbox (2) Most Negative:  (0)
Most Postive: #Xbox (2) Most Negative:  (0)
Most Postive: #Xbox (2)

KeyboardInterrupt: 